### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
5785,2023-11-08,Brasil Nbb,19:30,São José,Minas,3.94,1.24,162.5,1.88,1.88,8.5,1.97,1.74
5786,2023-11-08,Brasil Nbb,20:00,São Paulo,Unifacisa,1.56,2.38,154.5,1.87,1.89,-4.5,1.98,1.74
5787,2023-11-08,Eua Nba,21:00,Charlotte Hornets,Washington Wizards,1.65,2.44,238.5,1.83,2.05,-3.5,2.05,1.89
5788,2023-11-08,Eua Nba,21:00,Indiana Pacers,Utah Jazz,1.42,3.23,241.5,1.83,2.05,-6.5,1.97,1.97
5789,2023-11-08,Eua Nba,21:00,Philadelphia 76ers,Boston Celtics,2.15,1.82,225.5,1.82,2.11,1.5,2.05,1.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,2023-11-08,Rússia Liga Vtb United,13:00,Nizhny Novgorod,Saratov,1.47,2.75,159.5,1.86,1.88,-6.5,2.04,1.71
5873,2023-11-08,Rússia Liga Vtb United,13:30,Zenit Petersburg,Parma Perm,1.19,4.88,153.5,1.81,1.95,-11.5,2.07,1.69
5874,2023-11-08,Rússia Liga Vtb United,14:00,CSKA Moscow,Samara,1.12,6.45,161.5,1.83,1.95,-14.5,2.04,1.73
5875,2023-11-08,Suécia Superettan,15:00,RIG Mark,Hogsbo,2.53,1.47,149.5,1.83,1.83,3.5,2.00,1.72


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,21:00,Eua Nba,Charlotte Hornets,Washington Wizards,238.5,1.83,1.0000,Over
3,21:00,Eua Nba,Indiana Pacers,Utah Jazz,241.5,1.83,1.0000,Over
4,21:00,Eua Nba,Philadelphia 76ers,Boston Celtics,225.5,1.82,1.0000,Over
5,21:30,Eua Nba,Brooklyn Nets,Los Angeles Clippers,226.5,1.82,1.0000,Over
6,21:30,Eua Nba,New York Knicks,San Antonio Spurs,222.5,1.86,1.0000,Over
7,22:00,Eua Nba,Chicago Bulls,Phoenix Suns,220.5,1.86,1.0000,Over
8,22:00,Eua Nba,Houston Rockets,Los Angeles Lakers,217.5,1.86,1.0000,Over
9,22:00,Eua Nba,Memphis Grizzlies,Miami Heat,218.5,1.86,1.0000,Over
10,22:00,Eua Nba,Milwaukee Bucks,Detroit Pistons,226.5,1.86,1.0000,Over
11,22:00,Eua Nba,Minnesota Timberwolves,New Orleans Pelicans,217.5,1.80,1.0000,Over
